In [266]:
import numpy as np
import pandas as pd
import json
from rauth import OAuth1Service,OAuth1Session
import requests 
import webbrowser
import datetime
from requests_oauthlib import OAuth1
import time

consumer_key='85bdd7b968b11348fd8682f33a1d1c82'
consumer_secret='1fd4678dd9db8eefeb9046bfcfda129d'

positions = pd.read_csv('overbought_data.csv',index_col='symbol')
positions['open_date'] = pd.to_datetime(positions['open_date'],infer_datetime_format=True)
positions['open_lenth'] = datetime.datetime.now().date() - positions['open_date']

all_trades_df = pd.read_csv('all_trades.csv',index_col='Unnamed: 0')

is_long = False
is_long_2 = False
is_short = False
spy_data_dict = {'times':[],'prices':[]}

etrade = OAuth1Service(
    name="etrade",
    consumer_key='85bdd7b968b11348fd8682f33a1d1c82',
    consumer_secret='1fd4678dd9db8eefeb9046bfcfda129d',
    request_token_url="https://api.etrade.com/oauth/request_token",
    access_token_url="https://api.etrade.com/oauth/access_token",
    authorize_url="https://us.etrade.com/e/t/etws/authorize?key={}&token={}",
    base_url="https://api.etrade.com")

request_token, request_token_secret = etrade.get_request_token(
        params={"oauth_callback": "oob", "format": "json"})

authorize_url = etrade.authorize_url.format(etrade.consumer_key, request_token)
webbrowser.open(authorize_url)
text_code = input("Please accept agreement and enter text code from browser: ")
session = etrade.get_auth_session(request_token,
                                  request_token_secret,
                                  params={"oauth_verifier": text_code})

accounts_list_url = "https://api.etrade.com/v1/accounts/list.json"
r = session.get(accounts_list_url)
# r = requests.get(accounts_list_url,auth=auth)
r = json.loads(r.text)
rollover_key = r['AccountListResponse']['Accounts']['Account'][1]['accountIdKey']

def get_data(symbol):
    
    r = session.get(f'https://api.etrade.com/v1/market/quote/{symbol}.json')
    r = json.loads(r.content)
    market_price = round((r['QuoteResponse']['QuoteData'][0]['All']['ask']+r['QuoteResponse']['QuoteData'][0]['All']['bid'])/2,2)
    curr_time = int(str(datetime.datetime.now().month)+str(datetime.datetime.now().day)+str(datetime.datetime.now().hour)+str(datetime.datetime.now().minute)+str(datetime.datetime.now().second))
    return market_price,curr_time

opening_symbols = input('Comma separated list of tickers today: ').split(',')

hour = datetime.datetime.now().hour
minute = datetime.datetime.now().minute
second = datetime.datetime.now().second

while hour > 6 and hour < 8: 
    if second == 0 :
        
        for symbol in positions.index: 
    
            market_price, curr_time = get_data(symbol)
        
            if ((market_price - positions.loc[symbol]['trade_price'])/positions.loc[symbol]['trade_price']) >=1: 

                trade_data = {'symbol':symbol, 'date':datetime.datetime.now(),'trade_price':market_price,'side':'B'}

                close_positions_info.append(trade_data)

                positions.drop(symbol,inplace=True)
                
        time.sleep(57)
        second = datetime.datetime.now().second
        hour = datetime.datetime.now().hour
        minute = datetime.datetime.now().minute
                
    else:
        time.sleep(0.2)
        second = datetime.datetime.now().second
        hour = datetime.datetime.now().hour
        minute = datetime.datetime.now().minute
    
        
        
print('Between 8 and 8:30')

while hour == 8 and minute < 30:
    if second == 0 :
        time.sleep(1)
        second = datetime.datetime.now().second
        hour = datetime.datetime.now().hour
        minute = datetime.datetime.now().minute
    else:
        time.sleep(0.2)
        second = datetime.datetime.now().second
        hour = datetime.datetime.now().hour
        minute = datetime.datetime.now().minute
        
print('8:30')

open_positions_info = []
close_positions_info = []

for symbol in opening_symbols: 
    
    market_price, curr_time = get_data(symbol)
    
    trade_data = {'symbol':symbol, 'date':str(datetime.datetime.now().date()),'trade_price':market_price, 'side':'S'}
    
    open_positions_info.append(trade_data)
    
    time.sleep(1)
    
trade_2 = False
trade_3 = False

last_five_minute = datetime.datetime.now().minute - 1

while hour <11:
    curr_min = datetime.datetime.now().minute
    
    if hour == 9 and minute == 30 and not trade_2:
        
        for symbol in opening_symbols:
            market_price, curr_time = get_data(symbol)

            trade_data = {'symbol':symbol, 'date':str(datetime.datetime.now().date()),'trade_price':market_price,'side':'S'}

            open_positions_info.append(trade_data)
            
            trade_2 = True
            
        time.sleep(10)
        
    if hour == 10 and minute == 0 and not trade_3: 
        
        for symbol in opening_symbols:
            market_price, curr_time = get_data(symbol)

            trade_data = {'symbol':symbol, 'date':str(datetime.datetime.now().date()),'trade_price':market_price,'side':'S'}

            open_positions_info.append(trade_data)
            
            trade_3 = True
            
        time.sleep(10)
    
    if minute % 5 == 0 and curr_min != last_five_minute:
        
        for symbol in positions.index: 

            market_price, curr_time = get_data(symbol)

            if ((market_price - positions.loc[symbol]['trade_price'])/positions.loc[symbol]['trade_price']) >=1: 

                trade_data = {'symbol':symbol, 'date':datetime.datetime.now(),'trade_price':market_price,'side':'B'}

                close_positions_info.append(trade_data)

                positions.drop(symbol,inplace=True)
        
            print(minute,market_price)
            
        last_five_minute = datetime.datetime.now().minute

    time.sleep(10)
    second = datetime.datetime.now().second
    hour = datetime.datetime.now().hour
    minute = datetime.datetime.now().minute
    
open_df = pd.DataFrame(open_positions_info)
    
df = pd.DataFrame(open_positions_info)
df['open_date'] = df['date']
df = df.groupby(['symbol','open_date'], as_index=False)['trade_price'].mean()
df.set_index('symbol',inplace=True)
positions = positions.append(df)
positions['open_date'] = pd.to_datetime(positions['open_date'],infer_datetime_format=True)
positions['open_lenth'] = datetime.datetime.now().date() - positions['open_date']
    
while hour < 15:
    
    for symbol in positions.index: 
        
        market_price, curr_time = get_data(symbol)
        
        if ((market_price - positions.loc[symbol]['trade_price'])/positions.loc[symbol]['trade_price']) >=1: 
            
            trade_data = {'symbol':symbol, 'date':datetime.datetime.now(),'trade_price':market_price,'side':'B'}
            
            close_positions_info.append(trade_data)
            
            positions.drop(symbol,inplace=True)
            
            print(symbol,close_positions_info)
          
    time.sleep(10)
    second = datetime.datetime.now().second
    hour = datetime.datetime.now().hour
    minute = datetime.datetime.now().minute

close_df = pd.DataFrame(close_positions_info)
                  
all_trades_df = all_trades_df.append(open_df)
all_trades_df = all_trades_df.append(close_df)
                  
all_trades_df.to_csv('all_trades.csv')

positions.to_csv('overbought_data.csv')
   
            
        


Please accept agreement and enter text code from browser: Q3A6V
Comma separated list of tickers today: PED
Between 8 and 8:30
8:30
30 2.58
30 1.98
30 348.38
30 508.68
35 2.58
35 1.92
35 347.9
35 508.64
40 2.55
40 2.05
40 347.78
40 507.89
45 2.54
45 2.05
45 348.21
45 508.3
50 2.54
50 2.05
50 348.43
50 508.36
55 2.54
55 2.05
55 349.12
55 507.62
0 2.52
0 2.06
0 349.32
0 507.24
5 2.51
5 2.06
5 348.62
5 504.67
10 2.51
10 2.0
10 347.81
10 503.71
15 2.42
15 1.96
15 347.82
15 504.35
20 2.38
20 1.94
20 348.5
20 505.33
25 2.4
25 2.0
25 348.69
25 504.34
30 2.43
30 2.01
30 349.11
30 505.49
35 2.44
35 2.03
35 349.5
35 505.48
40 2.42
40 2.05
40 349.37
40 504.42
45 2.43
45 2.06
45 349.02
45 504.18
50 2.44
50 2.08
50 348.8
50 503.26
55 2.44
55 2.12
55 349.35
55 504.48
0 2.44
0 2.06
0 349.62
0 504.82
5 2.44
5 2.09
5 349.69
5 505.29
10 2.44
10 2.07
10 349.49
10 505.09
15 2.42
15 2.09
15 349.27
15 504.96
20 2.44
20 2.08
20 348.91
20 504.17
25 2.44
25 2.08
25 349.06
25 503.77
30 2.46
30 2.08
30 349.04
30 

KeyboardInterrupt: 

In [272]:
for symbol in positions.index: 
        
    market_price, curr_time = get_data(symbol)

    print((market_price - positions.loc[symbol]['trade_price'])/positions.loc[symbol]['trade_price'])

-0.36482939632545935
-0.04245283018867939
0.0014657699603379632
-0.007127625967461723
-1.0


In [268]:
positions

,open_date,open_lenth,trade_price
symbol,,,
SGLB,2020-08-25,2 days,3.810000
WORX,2020-08-26,1 days,2.120000
SPY,2020-08-26,1 days,347.940000
AAPL,2020-08-26,1 days,506.480000
PED,2020-08-27,0 days,1.666667


,date,side,symbol,trade_price
0,2020-08-27,S,PED,1.52
1,2020-08-27,S,PED,1.69
2,2020-08-27,S,PED,1.79
